In [ ]:

import sys 
import os
from datetime import datetime
import pandas as pd
import numpy as np

sys.path.append(os.path.abspath(os.getcwd()))

import pooldata as p_data
import JIT_funcs as JIT

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [65]:
#Parameters of the case to analyze
begin=datetime(2022,6,1)
end=datetime(2022,8,7)
pools=p_data.query_top_pools(900,'volumeUSD')
pools['pool']=pools['pool_name'].str.split("-").str[0:2]
#Keeping only ETH/Stables pools
# pools=pools.loc[pools['pool'].isin([['USDC','WETH'],['WETH','USDT'],['DAI','WETH']])]
pools_adresses=pools['id'].to_list()

In [63]:
pools

,id,createdAtTimestamp,feeTier,volumeUSD,token0.symbol,token0.decimals,token0.derivedETH,token1.symbol,token1.decimals,token1.derivedETH,pool_name,pool
0,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,1620250931,500,2271140331355228.381396634627922874,USDC,6,1355.560292350832307696482252117629,WETH,18,1,USDC-WETH-500,"[USDC, WETH]"
1,0x3416cf6c708da44db2624d63ea0aaef7113527c6,1636825557,100,1388253662125011.10806676365316927,USDC,6,1355.560292350832307696482252117629,USDT,6,1355.698549747533428121813954520138,USDC-USDT-100,"[USDC, USDT]"
2,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,1620246230,500,353034679825688.8016893125483549686,WBTC,8,22983712.68348797282148546461275297,WETH,18,1,WBTC-WETH-500,"[WBTC, WETH]"
3,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,1620169800,3000,220775700940932.6015045656399362049,USDC,6,1355.560292350832307696482252117629,WETH,18,1,USDC-WETH-3000,"[USDC, WETH]"
4,0x11b815efb8f581194ae79006d24e0d814b7697f6,1620251172,500,214236378416004.4674977612142320331,WETH,18,1,USDT,6,1355.698549747533428121813954520138,WETH-USDT-500,"[WETH, USDT]"
...,...,...,...,...,...,...,...,...,...,...,...,...
895,0x824664650c789bf01b5bba80473977ca820fb1eb,1634413112,10000,62039493.91208518105857777590865451,EJS,18,0.00001884146840288591901811768834757014,WETH,18,1,EJS-WETH-10000,"[EJS, WETH]"
896,0x5e2474ef1596acec4a7db53e7f666db1d1132afb,1629908290,10000,61899431.03546977421982786018263641,WETH,18,1,PEBBLE,18,0.00000003034868333747385588929386336072705,WETH-PEBBLE-10000,"[WETH, PEBBLE]"
897,0xb7c90598ccdaebcd14f43fa6635c422c3c5cecca,1621600172,3000,61603315.7653882460533001342916385,FEI,18,1352.350610761252097428749356046213,USDT,6,1355.698549747533428121813954520138,FEI-USDT-3000,"[FEI, USDT]"
898,0x842d60d060cef12c2a97706c2794b4ca96353d66,1636806949,3000,60944354.385831,SPELL,18,0.000000498063296181412132210332201092055,USDC,6,1355.560292350832307696482252117629,SPELL-USDC-3000,"[SPELL, USDC]"


In [ ]:
#Loop to get a dataframe with all JIT operations happened in the given dataframe for every pool in the list
# 寻找一个合适的池子来分析，因为the graph上一些小池子的数据不是很全面，所以要找一个数据全的
for idx,pool_adress in enumerate(pools_adresses):
    
    #Handling pools with no JIT operations with a try/except
    try:
        
        #Using JIT_funcs to obtain all JIT operations
        jit_final=JIT.JIT_pool(begin,end,pool_adress)
        
        #Using poolDayData to obtain the total fees generated by the pool
        pool_data=p_data.extract_pooldayData(begin,end,pool_adress)
        pool_data['feesUSD']=pool_data['feesUSD'].astype(float)
        pool_data['feesUSD_ac_pool']=pool_data['feesUSD'].cumsum()
        pool_data['date']=pool_data['timestamp_1'].dt.date
        jit_final['date']=jit_final['timestamp_1'].dt.date

        #Merging pool data with JIT data in order to compare it
        jit_final=jit_final.merge(pool_data[['date','feesUSD_ac_pool']],how='left',left_on='date',right_on='date')

        #Ratio of fees extracted by JIT operations
        ratio_extracted=jit_final['feesUSD'].sum()/pool_data['feesUSD'].sum()
        
        #Storing data with a readable pool_id
        pool_name=pools['pool_name'].iloc[idx]
        jit_final['pool_name']=pool_name
        #Saving to a csv
        jit_final.to_csv(path_result+"\\%s.csv" %(pool_adress))
        #Creating a summary in a dict
        results[pool_name]={
            'feesUSD': pool_data['feesUSD'].sum() ,
            'feesExtracted':jit_final['feesUSD'].sum() ,
            'ratio_extracted': round(ratio_extracted,5) ,
        }
        print(results)

    except: 
        print("%s doesnt have JIT operations" %(pools['pool_name'].iloc[idx]))
        results[pools['pool_name'].iloc[idx]]={ 'ratio_extracted': 'No JIT operations' }
        pass

In [ ]:
# 随便拿一个池子进行分析 这个是usdc和eth的池子地址
pool_adress = "0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640"
pool_data=p_data.extract_pooldayData(begin,end,pool_adress)
pool_data

In [29]:
pool_data['feesUSD']=pool_data['feesUSD'].astype(float)
pool_data['feesUSD_ac_pool']=pool_data['feesUSD'].cumsum()
pool_data['date']=pool_data['timestamp_1'].dt.date

1       18101.5785303805129733040755780067
2      7157.984063712348590419121322310211
3      9959.180090630325493068984369249912
4      13411.43750534422300944738797215834
5      10196.81880820284160005648448992291
                      ...                 
182    13376.81664760783499259364951945921
183    20953.99298477595600222498507777986
184    6275.647391457925121129859884486456
185    5762.782701510337318258562511721974
186    5091.623008089733991129476182154829
Name: feesUSD, Length: 186, dtype: object


In [ ]:
pool_data['feesUSD']=pool_data['feesUSD'].astype(float)
pool_data['feesUSD_ac_pool']=pool_data['feesUSD'].cumsum()
pool_data['date']=pool_data['timestamp_1'].dt.date
jit_final['date']=jit_final['timestamp_1'].dt.date